In [1]:

# 示例

import langextract as lx
import textwrap

prompt = textwrap.dedent("""\
    从报告文本中按出现顺序抽取 basic_info、clinical_info、epidemiological_info 三类实体。
    使用原文精确片段作为 extraction_text，不要意译或重组。
    每个实体的 attributes 按 schema 填写，可缺省但不得虚构。
""")

examples = [
    lx.data.ExampleData(
        text="""采访者：您好…现在住在哪里？
刘淑叶：我叫刘淑叶，今年73岁，是个农民，现在住在商州区杨峪河镇下赵塬移民搬迁小区。
采访者：之前有没有不舒服？
刘淑叶：血糖高大概2年。
采访者：现在在哪？情况如何？
刘淑叶：在商洛市中心医院感染二科住院，病情平稳。
采访者：最近有没有被蜱虫叮咬过？有没有野外活动史？
刘淑叶：一周前在小区后面的山上打枣，有只蜱虫叮在了我的腿上，当时我就拔了。
""",
        extractions=[
            lx.data.Extraction(
                extraction_class="basic_info",
                extraction_text="刘淑叶：我叫刘淑叶，今年73岁，是个农民，现在住在商州区杨峪河镇下赵塬移民搬迁小区。",
                attributes={
                    "name": "刘淑叶",
                    "gender": "女",
                    "age": "73",
                    "occupation": "农民",
                    "address": "商州区杨峪河镇下赵塬移民搬迁小区"
                }
            ),
            lx.data.Extraction(
                extraction_class="clinical_info",
                extraction_text="刘淑叶：血糖高大概2年。\n采访者：现在在哪？情况如何？\n刘淑叶：在商洛市中心医院感染二科住院，病情平稳。",
                attributes={
                    "hospital": "商洛市中心医院感染二科",
                    "diagnosis": None,
                    "symptoms": ["血糖高"],
                    "onset_date": None,
                    "status": "病情平稳"
                }
            ),
            lx.data.Extraction(
                extraction_class="epidemiological_info",
                extraction_text="刘淑叶：一周前在小区后面的山上打枣，有只蜱虫叮在了我的腿上，当时我就拔了。",
                attributes={
                    "exposure_history": "野外活动（打枣时被蚊虫叮咬）",
                    "contact_with_patients": "否",
                    "travel_history": "无外出"
                }
            ),
        ]
    )
]


In [2]:

# 模型构建


import os
from langextract import factory

api_key=os.getenv("DASHSCOPE_API_KEY") 

# When you need explicit provider selection or advanced configuration
from langextract import factory

# Specify both model and provider (useful when multiple providers support same model)
config = factory.ModelConfig(
    model_id="qwen-plus",
    provider="OpenAILanguageModel",  # Explicitly use Ollama
    provider_kwargs={
        "api_key": api_key,
        "base_url": "https://dashscope.aliyuncs.com/compatible-mode/v1",  # 关键！
    }
)
model = factory.create_model(config)


In [3]:

# 提取结果

# The input text to be processed
input_text = """
采访者：请您介绍一下基本情况和现在住在哪里？
张桂兰：我叫张桂兰，女，58岁，个体经营者，常住在商州区夜村镇朱家台村。
采访者：最近身体哪里不舒服？什么时候开始的？
张桂兰：从上周日开始发热、头痛、乏力，右小腿有一处红斑，还有点恶心，没有咳嗽。
采访者：目前在哪里就诊？医生怎么说？
张桂兰：今天到商洛市中心医院急诊就诊，医生说考虑是蜱传相关感染，先做血常规和肝功能，血小板偏低，让我留观。
采访者：最近有没有外出或野外活动？有没有被蜱虫叮咬的情况？
张桂兰：上周六我上山砍柴，回家洗澡时发现右小腿外侧有一只小黑虫叮住，后来用镊子夹掉了，皮肤那里有个小红点。
采访者：近期是否外地旅行？身边是否有发热病人或类似症状者？家里有养动物吗？
张桂兰：没有外地旅行，家里养了一条狗，最近经常跟着我上山；身边没听说有发热病人。
"""

# Run the extraction
result = lx.extract(
    text_or_documents=input_text,
    prompt_description=prompt,
    examples=examples,
    model=model,
    fence_output=True,
    use_schema_constraints=False,
)


In [4]:

# 保存结果

print(repr(result.text[:200]))

# Save the results to a JSONL file
lx.io.save_annotated_documents([result], output_name="extraction_results.jsonl", output_dir=".")

# Generate the visualization from the file
html_content = lx.visualize("extraction_results.jsonl")
display(html_content) #Jupyter Notebook / IPython 内核提供的一个 通用显示接口。能“智能识别”传入对象的类型，并调用相应的“富媒体展示方法”把结果显示在 Notebook 单元格输出区。
with open("visualization.html", "w", encoding="utf-8") as f:
    # ✅ 先取出字符串内容
    if hasattr(html_content, 'data'):
        html_str = html_content.data  # For Jupyter/Colab
    else:
        html_str = html_content

    # ✅ 补上 meta charset，防止浏览器默认用 GBK 打开导致乱码
    if "<meta charset" not in html_str.lower():
        html_str = "<meta charset='utf-8'>\n" + html_str

    # ✅ 写入文件
    f.write(html_str)



'\n采访者：请您介绍一下基本情况和现在住在哪里？\n张桂兰：我叫张桂兰，女，58岁，个体经营者，常住在商州区夜村镇朱家台村。\n采访者：最近身体哪里不舒服？什么时候开始的？\n张桂兰：从上周日开始发热、头痛、乏力，右小腿有一处红斑，还有点恶心，没有咳嗽。\n采访者：目前在哪里就诊？医生怎么说？\n张桂兰：今天到商洛市中心医院急诊就诊，医生说考虑是蜱传相关感染，先做血常规和肝功能，血小板偏低，让我留观。\n采访者'


LangExtract: Saving to extraction_results.jsonl: 1 docs [00:00, 249.72 docs/s]

✓ Saved 1 documents to extraction_results.jsonl



LangExtract: Loading extraction_results.jsonl: 100%|██████████████████████████████▉| 3.17k/3.17k [00:00<00:00, 147kB/s]

✓ Loaded 1 documents from extraction_results.jsonl


In [5]:

# 可视化结果

from IPython.display import FileLink

# 显示一个可点击的下载/打开链接
FileLink("visualization.html")


C:\Users\传防科电脑\visualization.html

In [ ]:

# 保存结果到json

import json

# 读取保存的 JSONL 文件
with open("extraction_results.jsonl", "r", encoding="utf-8") as f:
    # jsonl 格式是逐行 JSON，这里只取第一行
    line = f.readline()
    extraction_results = json.loads(line)

# 漂亮打印
print(json.dumps(extraction_results, indent=2, ensure_ascii=False))
